## Find the number of people vaccinated with 1 or 2 doses of any vaccine

In [1]:
import numpy as np
import pandas as pd
import datetime as dt

In [2]:
# from datetime import datetime
# dateparse = lambda x: datetime.strptime(x, '%d/%m/%Y')

In [3]:
#READING DOSES FILE
df = pd.read_csv(r'firstandseconddoselatestnew1.csv',parse_dates=['Date'])

In [4]:
# df.head(10)

# DISTRICT WEEKWISE

In [5]:
# COPY OF DATAFRAME
newdf = df.copy()
#SETTING DATE COLUMNS AS INDEX
newdf.index

RangeIndex(start=0, stop=153819, step=1)

In [6]:
#A DATAFRAME TO STORE EACH DISTRICT
col_df = pd.DataFrame()

In [7]:
def thefunc_week(name):
    dist_df = pd.DataFrame()
    dfsub = newdf.loc[newdf['District_Key']==name]
    firstdose = []
    seconddose = []

    d1 = list(dfsub['dose1'])
    d2 = list(dfsub['dose2'])
    
    #appending 16th date as it is
    firstdose.append(d1[0])
    seconddose.append(d2[0])
    
    firstindex,lastindex = 1,0
    firstval1 = d1[0]
    firstval2 = d2[0]
    lastval1=0
    lastval2=0
    
    seconddoseval = d2[0]
    

    for i in range(28):
        #getting last value of week
        lastval1 = d1[firstindex:firstindex+7][-1] 
        lastval2 = d2[firstindex:firstindex+7][-1]
        
        #diff which gives present week sum
        diff1 = lastval1  - firstval1
        diff2 = lastval2  - firstval2

        #appending to respective list
        firstdose.append(diff1)
        seconddose.append(diff2)
        #d.append(lastval1)

        #changing last week value
        firstval1 = lastval1
        firstval2 = lastval2

        firstindex = firstindex+7


    #last weeek val doubt    
    l1 = d1[-1]-lastval1
    l2 = d2[-1]-lastval2
    firstdose.append(l1)
    seconddose.append(l2)
    

    week =[i for i in range(1,31)]
    dist_df['weeklydose1'] = firstdose
    dist_df['weeklydose2'] = seconddose
    dist_df['District_Key'] = name
    dist_df['week'] = week

    
    return dist_df   #Returns individual district datframe

In [8]:
#DISTRICT_names
dist_names = newdf['District_Key'].unique()

In [9]:
# COLLECTION OF ALL DISTRICTS IN A LIST
coll_df = []              #

In [10]:
for i in range(len(dist_names)):
#     print(dist_names[i])

    st=thefunc_week(dist_names[i])   #st is --each district id dataframe
    coll_df.append(st)                  #appending to a list

In [11]:
# CONVERTING LIS OF DATAFRAMES INTO A SINGLE DATAFRAME
finaldf_week = pd.concat(coll_df)
finaldf_week.reset_index(inplace=True,drop=True)

In [12]:
#RENAMING COLUMNS
finaldf_week.rename(columns={'District_Key':'districtid','week':'timeid','weeklydose1':'dose1','weeklydose2':'dose2'},inplace=True)
finaldf_week = finaldf_week[['districtid','timeid','dose1','dose2']]

In [13]:
#checking
finaldf_week.to_csv('district-vaccinated-count-week.csv',index=False)
# district-vaccinated-count-weekly

In [14]:
#FINAL OUTPUT FILE
# finaldf_week.to_csv('Districtweekques5.csv',index=False)

In [15]:
# keep first date as its
# run for loop 29 times

# DISTRICT MONTHLY

In [16]:
mdates = [['2021-01-16','2021-02-14'],['2021-02-15','2021-03-14'],['2021-03-15','2021-04-14'],
          ['2021-04-15','2021-05-14'],['2021-05-15','2021-06-14'],['2021-06-15','2021-07-14'],['2021-07-15','2021-08-14']]

In [17]:
for i in range(len(mdates)):
    mdates[i][0]=pd.to_datetime(mdates[i][0])
    mdates[i][1]=pd.to_datetime(mdates[i][1])

In [18]:
len(mdates)
type(mdates[0][0])
mdates

[[Timestamp('2021-01-16 00:00:00'), Timestamp('2021-02-14 00:00:00')],
 [Timestamp('2021-02-15 00:00:00'), Timestamp('2021-03-14 00:00:00')],
 [Timestamp('2021-03-15 00:00:00'), Timestamp('2021-04-14 00:00:00')],
 [Timestamp('2021-04-15 00:00:00'), Timestamp('2021-05-14 00:00:00')],
 [Timestamp('2021-05-15 00:00:00'), Timestamp('2021-06-14 00:00:00')],
 [Timestamp('2021-06-15 00:00:00'), Timestamp('2021-07-14 00:00:00')],
 [Timestamp('2021-07-15 00:00:00'), Timestamp('2021-08-14 00:00:00')]]

In [19]:
# COPY OF DATAFRAME
newdf = df.copy()
newdf.set_index('Date',inplace=True)
# newdf.index

In [20]:
def thefunc(name):
    dfsub = newdf.loc[newdf['District_Key']==name]
    #AS DATA IS CUMULATIVE WE USE LAST ROW OF EACH DISTRICT
    def first_last(df1):
        return df1.iloc[[-1]]
    
    dfsub1= dfsub[0:0]  #creating an empty dataframe
    dfsub1['Month'] = 0  #ASSGNINS MONTH ID
    
    ls=[]
    for i in range(len(mdates)):
        try:
            row = (dfsub[mdates[i][0]:mdates[i][1]][-1:])
            d = pd.DataFrame(row)
            if len(d.index) == 0:
                continue
            dfsub1 = pd.concat([dfsub1,d])
            ls.append(i+1)
        except:
            continue
    dfsub1['Month'] = ls    #HERE MONTHIDS ARE ASSGNED FROM THE LIST
    
    #AS "DIFF"func CREATES NAN IN 1stROW ,TO OVERCOME THIS WE STORE VALUES IN VARIABLES AND PLACE INTO 1ST ROW AT THE END
    val1= dfsub1['dose1'][0]
    val2= dfsub1['dose2'][0]
    
    #just creating a copy
    dfsub2 = dfsub1.copy()
    dfsub2['dose1'] = dfsub1.dose1.diff()
    dfsub2['dose2'] = dfsub1.dose2.diff()
    
    dfsub2['dose1'].fillna(value=val1,inplace=True)
    dfsub2['dose2'].fillna(value=val2,inplace=True)

    
    return dfsub2

In [21]:
#DISTRICT_names
dist_names = newdf['District_Key'].unique()

In [22]:
# COLLECTION OF ALL DISTRICTS IN A LIST
coll_df2 = []              

In [23]:
for i in range(len(dist_names)):
#     print(dist_names[i])
    st=thefunc(dist_names[i])   #each district id dataframe
    coll_df2.append(st)
    
print('end')


end


In [24]:
# CONVERTING LIST OF DATAFRAMES INTO A SINGLE DATAFRAME
finaldf2 = pd.concat(coll_df2)
finaldf2.reset_index(inplace=True,drop=True)

In [25]:
finaldf2 = finaldf2.sort_values(['District_Key','Month'])

In [26]:
#RENAMING COLUMNS
finaldf2.rename(columns={'District_Key':'districtid','Month':'timeid'},inplace=True)
finaldf2 = finaldf2[['districtid','timeid','dose1','dose2']]

In [27]:
#FINAL OUTPUT
finaldf2.to_csv('district-vaccinated-count-month.csv',index = False)
# district-vaccinated-count-monthly

# DISTRICT OVERALL

In [28]:
def thefunc_overall(name):
    dfsub = newdf.loc[newdf['District_Key']==name]
    return dfsub.iloc[[-1]]                    #ASDATA IS CUMULATIVE
    


In [29]:
coll_df5 = []         #LIST OF ALL DISTRICTS

In [30]:
# CALCULATING FOR EACH DISTRICT
for i in range(len(dist_names)):
    st=thefunc_overall(dist_names[i])   
    coll_df5.append(st)

In [31]:
#COLLECTION OF ALL DISTRICTS INTO A SINGLE DATAFRAME
finaldf5 = pd.concat(coll_df5)
finaldf5.reset_index(inplace=True,drop=True)

In [32]:
finaldf5['overall'] = 1  #ADDING TIMEID

In [33]:
finaldf5.rename(columns={'District_Key':'districtid','overall':'timeid'},inplace=True)
finaldf5 = finaldf5[['districtid','timeid','dose1','dose2']]
finaldf5

,districtid,timeid,dose1,dose2
0,AN_Nicobars,1,22081.0,8110.0
1,AN_North and Middle Andaman,1,68043.0,29281.0
2,AN_South Andaman,1,142585.0,59896.0
3,AP_Anantapur,1,1364374.0,597346.0
4,AP_Chittoor,1,1565591.0,640864.0
...,...,...,...,...
724,WB_Purba Bardhaman,1,937024.0,457682.0
725,WB_Purba Medinipur,1,1539152.0,672014.0
726,WB_Purulia,1,580596.0,206567.0
727,WB_South 24 Parganas,1,2067595.0,793261.0


In [34]:
#WRITING INTO CSV
finaldf5.to_csv('district-vaccinated-count-overall.csv',index=False)

# STATE WEEKLY

In [35]:
#1ST WEEK CONSISTS OF ONLY JAN16 so we store into a variable
s_day = pd.to_datetime('2021/01/16')

#CONSISTS OF ALL WEEKDATES IN A LIST (SUN to SAT)
week_dates=[[s_day,s_day]]

In [36]:
weekstart = pd.date_range('2021-01-17','2021-08-14',freq='W-SUN')  #week starting dates
weekend = pd.date_range('2021-01-17','2021-08-14',freq='W-SAT')    #week ending dates

In [37]:
#APPEND INTO "WEEK_DATES" LIST
for i in range(len(weekstart)):
    week_dates.append([weekstart[i],weekend[i]])
    

In [38]:
week_dates[:5]

[[Timestamp('2021-01-16 00:00:00'), Timestamp('2021-01-16 00:00:00')],
 [Timestamp('2021-01-17 00:00:00', freq='W-SUN'),
  Timestamp('2021-01-23 00:00:00', freq='W-SAT')],
 [Timestamp('2021-01-24 00:00:00', freq='W-SUN'),
  Timestamp('2021-01-30 00:00:00', freq='W-SAT')],
 [Timestamp('2021-01-31 00:00:00', freq='W-SUN'),
  Timestamp('2021-02-06 00:00:00', freq='W-SAT')],
 [Timestamp('2021-02-07 00:00:00', freq='W-SUN'),
  Timestamp('2021-02-13 00:00:00', freq='W-SAT')]]

In [39]:
# COPY OF DATAFRAME
newdf = df.copy()
newdf.set_index('Date',inplace=True)


In [40]:
def thefunc_stweek(name):
    dfsub = newdf.loc[newdf['State']==name]
    
    #AS DATA IS CUMULATIVE WE USE LAST ROW OF EACH DISTRICT
    def first_last(df1):
        return df1.iloc[[-1]]
    
    dfsub1= dfsub[0:0]  #creating an empty dataframe which STORES EACH STATEDATA MONTHWISE (DFSUB1)
    dfsub1['week'] = 0
    
    ls=[]  #for weekids
    for i in range(len(week_dates)):
        
        #GROUPING DATA ON STATE AND SUMMING DOSES OF ALL DISTRICTS
        dfsub = dfsub.groupby(['State','Date']).aggregate({'dose1':sum,
                                                          'dose2':sum})
        dfsub = dfsub.reset_index()
        dfsub.set_index('Date',inplace=True)

        try:
            row = dfsub[week_dates[i][0]:week_dates[i][1]][-1:]       #COLLECTING THE  DATA IN THE RESPECTIVE MONTH
            d = pd.DataFrame(row)      

            if len(d.index) == 0:          #if no data present in a month
                continue
                
            dfsub1 = pd.concat([dfsub1,d])
            ls.append(i+1)
        except:
            continue

    dfsub1['week'] = ls       #HERE WEEKIDS ARE ASSGNED FROM THE LIST
    
    #AS "DIFF"func CREATES NAN IN 1stROW ,TO OVERCOME THIS WE STORE VALUES IN VARIABLES AND PLACE INTO 1ST ROW AT THE END
    val1= dfsub1['dose1'][0]
    val2= dfsub1['dose2'][0]
    
    dfsub2 = dfsub1.copy()
    dfsub2['dose1'] = dfsub1.dose1.diff()
    dfsub2['dose2'] = dfsub1.dose2.diff()
    
    dfsub2['dose1'].fillna(value=val1,inplace=True)
    dfsub2['dose2'].fillna(value=val2,inplace=True)
    
    return dfsub2

In [41]:
#UNIQUE STATE NAMES
unique_state_names = newdf['State'].unique()
coll_df3 = []              #apply some others func and append them to coll_df

In [42]:
for i in range(len(unique_state_names)):
#     print(unique_state_names[i])
    st=thefunc_stweek(unique_state_names[i])   #each district id dataframe
    coll_df3.append(st)
    
print('end')

end


In [43]:
# CONVERTING LIST OF DATAFRAMES INTO A SINGLE DATAFRAME
finaldf3 = pd.concat(coll_df3)
finaldf3.reset_index(inplace=True)

# finaldf3.head(20)

In [44]:
finaldf3 = finaldf3.sort_values(['State','week'])

In [45]:
#RENAMING COLUMNS
finaldf3.rename(columns={'State':'stateid','week':'timeid'},inplace=True)
finaldf3 = finaldf3[['stateid','timeid','dose1','dose2']]

In [46]:
#FINAL OUTPUT
finaldf3.to_csv('state-vaccinated-count-week.csv',index=False)
# state-vaccinated-count-weekly

# STATE MONTHLY

In [47]:
mdates_month = [['2021-01-16','2021-02-14'],['2021-02-15','2021-03-14'],['2021-03-15','2021-04-14'],
          ['2021-04-15','2021-05-14'],['2021-05-15','2021-06-14'],['2021-06-15','2021-07-14'],['2021-07-15','2021-08-14']]

In [48]:
for i in range(len(mdates_month)):
    mdates_month[i][0]=pd.to_datetime(mdates_month[i][0])
    mdates_month[i][1]=pd.to_datetime(mdates_month[i][1])

In [49]:
len(mdates_month)
type(mdates_month[0][0])
mdates_month

[[Timestamp('2021-01-16 00:00:00'), Timestamp('2021-02-14 00:00:00')],
 [Timestamp('2021-02-15 00:00:00'), Timestamp('2021-03-14 00:00:00')],
 [Timestamp('2021-03-15 00:00:00'), Timestamp('2021-04-14 00:00:00')],
 [Timestamp('2021-04-15 00:00:00'), Timestamp('2021-05-14 00:00:00')],
 [Timestamp('2021-05-15 00:00:00'), Timestamp('2021-06-14 00:00:00')],
 [Timestamp('2021-06-15 00:00:00'), Timestamp('2021-07-14 00:00:00')],
 [Timestamp('2021-07-15 00:00:00'), Timestamp('2021-08-14 00:00:00')]]

In [50]:
# COPY OF DATAFRAME
newdf = df.copy()
print(newdf.dtypes)
# newdf.set_index('Date',inplace=True)
newdf.index

State                   object
District                object
District_Key            object
dose1                  float64
dose2                  float64
Date            datetime64[ns]
dtype: object


RangeIndex(start=0, stop=153819, step=1)

In [51]:
def thefunc_stmonth(name):
    dfsub = newdf.loc[newdf['State']==name]
    
    def first_last(df1):
        return df1.iloc[[-1]]
    
    dfsub1= dfsub[0:0]  #creating an empty dataframe
    dfsub1['Month'] = 0
    
    ls=[]
    for i in range(len(mdates_month)):

        dfsub = dfsub.groupby(['State','Date']).aggregate({'dose1':sum,
                                                          'dose2':sum})

        dfsub = dfsub.reset_index()
        dfsub.set_index('Date',inplace=True)

        try:
            row = dfsub[mdates_month[i][0]:mdates_month[i][1]][-1:]

            
            d = pd.DataFrame(row)

            if len(d.index) == 0:
                continue
            dfsub1 = pd.concat([dfsub1,d])
            ls.append(i+1)
        except:
            continue

    dfsub1['Month'] = ls
    
    val1= dfsub1['dose1'][0]

    val2= dfsub1['dose2'][0]
    dfsub2 = dfsub1.copy()

    dfsub2['dose1'] = dfsub1.dose1.diff()
    dfsub2['dose2'] = dfsub1.dose2.diff()
    
    dfsub2['dose1'].fillna(value=val1,inplace=True)
    dfsub2['dose2'].fillna(value=val2,inplace=True)
    
    return dfsub2

In [52]:
unique_state_names = newdf['State'].unique()
# unique_state_names
coll_df4 = []              

In [53]:
count=0
for i in range(len(unique_state_names)):
#     print(unique_state_names[i])
    count=i
    st=thefunc_stmonth(unique_state_names[i])   #each district id dataframe
    coll_df4.append(st)
    
print('end')

end


In [54]:
finaldf4 = pd.concat(coll_df4)
finaldf4.reset_index(inplace=True)


In [55]:
finaldf4 = finaldf4.sort_values(['State','Month'])

In [56]:
#RENAMING COLUMNS
finaldf4.rename(columns={'State':'stateid','Month':'timeid'},inplace=True)
finaldf4 = finaldf4[['stateid','timeid','dose1','dose2']]

In [57]:
#FINAL OUTPUT
finaldf4.to_csv('state-vaccinated-count-month.csv',index=False)
# state-vaccinated-count-monthly

# STATE OVERALL

In [58]:
def thefunc_overall_st(name):
    dfsub = newdf.loc[newdf['State']==name]
    return dfsub.iloc[[-1]]                    #ASDATA IS CUMULATIVE

In [59]:
coll_df6 = []         #LIST OF ALL DISTRICTS

In [60]:
for i in range(len(unique_state_names)):
#     print(unique_state_names[i])
    st=thefunc_overall_st(unique_state_names[i])   #each district id dataframe
    coll_df6.append(st)
    
print('end')

end


In [61]:
#COLLECTION OF ALL DISTRICTS INTO A SINGLE DATAFRAME
finaldf6 = pd.concat(coll_df6)
finaldf6.reset_index(inplace=True,drop=True)

In [62]:
finaldf6['overall'] = 1  #ADDING TIMEID

In [63]:
#RENAMING COLUMNS
finaldf6.rename(columns={'State':'stateid','overall':'timeid'},inplace=True)
finaldf6 = finaldf6[['stateid','timeid','dose1','dose2']]

In [64]:
#FINAL OUTPUT
finaldf6.to_csv('state-vaccinated-count-overall.csv',index=False)